#  Taxi

In [1]:
from random import choice

In [3]:
import gym

#create a single game instance
env = gym.make("Taxi-v2")

#start new game
env.reset();

[2017-10-23 14:29:10,137] Making new env: Taxi-v2


In [4]:
# display the game state
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [5]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 406
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

observations: Discrete(500) n= 500
actions: Discrete(6) n= 6


In [6]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 406
reward: -1
is game over?: False
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [7]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [11]:
env.step(action_to_i['up'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


### Baseline: random search

### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [12]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 500 environment states.
    Element must be an integer from 0 to 5, representing action
    to take from that state.
    """
    return np.random.randint(0, n_actions, n_states)

In [13]:
print(n_states, n_actions)

500 6


In [14]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 500'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.1668264  0.1667818  0.166578   0.166587   0.166508   0.1667188]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [15]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for iteration in range(t_max):
        action = policy[s]
        s, r, done, _ = env.step(action)
        total_reward += r
        if done:
            break
    return total_reward

In [17]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env, get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [18]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

### Main loop

In [20]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)


  0%|          | 2/10000 [00:00<23:59,  6.94it/s]

New best score: -637.84



  0%|          | 4/10000 [00:00<23:02,  7.23it/s]

New best score: -592.3
New best score: -502.57


  0%|          | 11/10000 [00:01<20:13,  8.23it/s]

New best score: -456.67


  3%|▎         | 280/10000 [00:33<19:45,  8.20it/s]

New best score: -359.47


100%|██████████| 10000/10000 [21:23<00:00,  7.86it/s]


In [21]:
best_score

-359.47

# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [22]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    crossover_res = np.choose(np.random.choice(np.array([0,1]),
                                               size=policy1.shape, p=[p, 1-p]),
                              choices=[policy1, policy2])
    return crossover_res

In [23]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(get_random_policy(), policy, p)
    

In [24]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [25]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [26]:
print("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
pool_scores = [evaluate(policy) for policy in pool]


initializing...


In [27]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])


In [28]:
#main loop
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    crossovered = [crossover(choice(pool), choice(pool)) for _ in range(n_crossovers)]
    mutated = [mutation(choice(pool)) for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered + mutated #<add up old population with crossovers/mutations>
    pool_scores = [evaluate(policy) for policy in pool]#<evaluate all policies again>
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

Epoch 0:
best score: -404.83
Epoch 1:
best score: -448.03
Epoch 2:
best score: -440.2
Epoch 3:
best score: -430.39
Epoch 4:
best score: -413.2
Epoch 5:
best score: -396.01
Epoch 6:
best score: -403.57
Epoch 7:
best score: -386.38
Epoch 8:
best score: -358.93
Epoch 9:
best score: -386.29
Epoch 10:
best score: -341.47
Epoch 11:
best score: -368.38
Epoch 12:
best score: -377.38
Epoch 13:
best score: -351.01
Epoch 14:
best score: -324.28
Epoch 15:
best score: -341.29
Epoch 16:
best score: -332.74
Epoch 17:
best score: -323.02
Epoch 18:
best score: -323.92
Epoch 19:
best score: -296.65
Epoch 20:
best score: -322.93
Epoch 21:
best score: -296.47
Epoch 22:
best score: -270.37
Epoch 23:
best score: -270.28
Epoch 24:
best score: -288.46
Epoch 25:
best score: -252.19
Epoch 26:
best score: -198.64
Epoch 27:
best score: -225.1
Epoch 28:
best score: -225.37
Epoch 29:
best score: -242.65
Epoch 30:
best score: -252.64
Epoch 31:
best score: -216.37
Epoch 32:
best score: -207.73
Epoch 33:
best score: -